In [1]:
!pip install yfinance
!pip install streamlit
!pip install transformers
!pip install bs4
!pip install scikit_learn
!pip install hopsworks
!pip install seaborn
!pip install nltk

!pip install emoji
!pip install vaderSentiment

  Using cached streamlit-1.17.0-py2.py3-none-any.whl (9.3 MB)
  Using cached validators-0.20.0-py3-none-any.whl
  Using cached cachetools-5.2.1-py3-none-any.whl (9.3 kB)
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached GitPython-3.1.30-py3-none-any.whl (184 kB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached pydeck-0.8.0-py2.py3-none-any.whl (4.7 MB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Using cached pyarrow-10.0.1-cp39-cp39-macosx_11_0_arm64.whl (23.0 MB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached blinker-1.5-py2.py3-none-any.whl (12 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached tzdata-2022.7-py2.py3-none-any.whl (340 kB)
  Attempting uninstall: protobuf
    Found existing installation

In [ ]:
from data_loader_functions import *
from sentiments_analysis import *
from datetime import datetime, timedelta, timezone
import pandas as pd

## Upload news sentiments

In [ ]:
def timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

In [ ]:
apple_articles = sentiment_analysis('Apple', (datetime.today()))
apple_articles['date'] = apple_articles['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
apple_articles.date = apple_articles.date.apply(timestamp_2_time)

In [ ]:
apple_articles

In [ ]:
amazon_articles = sentiment_analysis('Amazon', (datetime.today()))
amazon_articles['date'] = amazon_articles['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
amazon_articles.date = amazon_articles.date.apply(timestamp_2_time)

In [ ]:
meta_articles = sentiment_analysis('Meta', (datetime.today()))
meta_articles['date'] = meta_articles['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
meta_articles.date = meta_articles.date.apply(timestamp_2_time)

## Upload stock price

In [45]:
apple_df = get_stock_price('AAPL', (datetime.today()), (datetime.today() + timedelta(days=1)))
apple_df


[*********************100%***********************]  1 of 1 completed


,name,date,open,high,low,close,adj_close,volume
0,APPLE,2023-01-05,127.129997,127.769997,124.760002,125.019997,125.019997,80962700


In [6]:
amazon_df = get_stock_price('AMZN', (datetime.today()), (datetime.today() + timedelta(days=1)))
amazon_df


[*********************100%***********************]  1 of 1 completed


,name,date,open,high,low,close,adj_close,volume
0,APPLE,2023-01-13,94.18,98.370003,94.120003,98.120003,98.120003,85413600


In [7]:
meta_df = get_stock_price('META', (datetime.today()), (datetime.today() + timedelta(days=1)))
meta_df


[*********************100%***********************]  1 of 1 completed


,name,date,open,high,low,close,adj_close,volume
0,META,2023-01-13,134.970001,137.389999,134.839996,136.979996,136.979996,22410700


In [22]:
meta_df['date'].dtype == object

False

In [10]:
frames = [meta_df, apple_df, amazon_df]
stocks_df = pd.concat(frames)
stocks_df['date'] = stocks_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
stocks_df.date = stocks_df.date.apply(timestamp_2_time)
stocks_df

,name,date,open,high,low,close,adj_close,volume
0,META,1673568000000,134.970001,137.389999,134.839996,136.979996,136.979996,22410700
0,APPLE,1673568000000,132.029999,134.919998,131.660004,134.759995,134.759995,57758000
0,APPLE,1673568000000,94.180000,98.370003,94.120003,98.120003,98.120003,85413600


In [11]:
stocks_df['price_move'] = -1.0

In [ ]:
stocks_df

,name,date,open,high,low,close,adj_close,volume,price_move
0,META,1673568000000,134.970001,137.389999,134.839996,136.979996,136.979996,22410700,-1.0
0,APPLE,1673568000000,132.029999,134.919998,131.660004,134.759995,134.759995,57758000,-1.0
0,AMAZON,1673568000000,94.180000,98.370003,94.120003,98.120003,98.120003,85413600,-1.0


In [15]:
stocks_df['date'].dtype

dtype('int64')

## Upload to Hopsworks

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5279
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
apple_fg = fs.get_feature_group(
        name = 'market_news_fg_Apple',
    )    
apple_fg.insert(apple_articles)

In [ ]:
amazon_fg = fs.get_feature_group(
        name = 'market_news_fg_Amazon',
    )    
amazon_fg.insert(amazon_articles)

In [ ]:
meta_fg = fs.get_feature_group(
        name = 'stocks_fg_Meta',
    )    

meta_fg.insert(meta_articles)

Uploading Dataframe: 0.00% |          | Rows 0/3 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5279/jobs/named/stocks_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fd495aa59a0>, None)